# Importing the libraries



In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import os
from nltk.corpus import wordnet
os.chdir('G:\MetaVerseCompany')
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import  hstack
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline 
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB


# Downloading the required sets such as stopwords or wordnet 

In [2]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaveh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kaveh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Importing the training data

In [4]:
train=pd.read_csv('train.csv')
train.head()



,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Identifying Missing Values

In [6]:
train.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

# Create a Class for Handling Missing Values (Used for Making PipeLine)

In [7]:
class Missing_Value(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Instance is Created')                
    def fit(self,Data):
        self.Data=Data
        self.Data['title'].fillna(value='Not Speciefied',inplace=True)
        self.Data['text'].fillna(value='Empty',inplace=True)
        self.Data['author'].fillna(value='unknown',inplace=True)
        return self
    def transform(self,Data,y=None):
        
        Data['title'].fillna(value='Not Speciefied',inplace=True)
        Data['text'].fillna(value='Empty',inplace=True)
        Data['author'].fillna(value='unknown',inplace=True)
        return Data
        
    def fit_transform(self,Data,y=None):
        self.Data=Data
        self.Data['title'].fillna(value='Not Speciefied',inplace=True)
        self.Data['text'].fillna(value='Empty',inplace=True)
        self.Data['author'].fillna(value='unknown',inplace=True)
        return self.Data


# Preprocessing the Text by Defining a Class (To be used in a Pipeline)

In [29]:
class preprocessing(BaseEstimator,TransformerMixin):
    def __init__(self):
        print('Instance is Created')                
    def fit(self,Data):
        self.length=Data.shape[0]
        self.Data=Data
        for i in range(self.length):
            cleaned_txt=self.clean_text(self.Data['text'].iloc[i])
            cleaned_title=self.clean_text(self.Data['title'].iloc[i])
            cleaned_text_splitted=cleaned_txt.split(' ')
            cleaned_title_splitted=cleaned_title.split(' ')

            words_and_tags_text=nltk.pos_tag(cleaned_text_splitted)
            words_and_tags_title=nltk.pos_tag(cleaned_title_splitted)
            text_parts=[]
            for word,tag in words_and_tags_text:
                lemmatizer = WordNetLemmatizer()
                word_=lemmatizer.lemmatize(word,pos=self.get_wordnet_part_of_speech(tag))
                text_parts.append(word_)

            title_parts=[]
            for word,tag in words_and_tags_title:
                lemmatizer = WordNetLemmatizer()
                word_=lemmatizer.lemmatize(word,pos=self.get_wordnet_part_of_speech(tag))
                title_parts.append(word_)


            self.Data['text'].iloc[i]=' '.join(text_parts)
            self.Data['title'].iloc[i]=' '.join(title_parts)

        return self
    def transform(self,Data,y=None):
        length=Data.shape[0]
       
        for i in range(length):
            cleaned_txt=self.clean_text(Data['text'].iloc[i])
            cleaned_title=self.clean_text(Data['title'].iloc[i])
            cleaned_text_splitted=cleaned_txt.split(' ')
            cleaned_title_splitted=cleaned_title.split(' ')

            words_and_tags_text=nltk.pos_tag(cleaned_text_splitted)
            words_and_tags_title=nltk.pos_tag(cleaned_title_splitted)
            text_parts=[]
            for word,tag in words_and_tags_text:
                lemmatizer = WordNetLemmatizer()
                word_=lemmatizer.lemmatize(word,pos=self.get_wordnet_part_of_speech(tag))
                text_parts.append(word_)

            title_parts=[]
            for word,tag in words_and_tags_title:
                lemmatizer = WordNetLemmatizer()
                word_=lemmatizer.lemmatize(word,pos=self.get_wordnet_part_of_speech(tag))
                title_parts.append(word_)


            Data['text'].iloc[i]=' '.join(text_parts)
            Data['title'].iloc[i]=' '.join(title_parts)
        return Data
    def fit_transform(self,Data,y=None):
        self.fit(Data)
        return self.Data
    def clean_text(self,text):
        text=text.lower()
        text=re.sub(r"i\'m",'i am',text)
        text=re.sub(r"i’m",'i am',text)

        text=re.sub(r"she\'s",'she is',text)
        text=re.sub(r"she’s",'she is',text)

        text=re.sub(r"that\'s",'that is',text)
        text=re.sub(r"that’s",'that is',text)

        text=re.sub(r"they\'re",'they are',text)
        text=re.sub(r"they’re",'they are',text)

        text=re.sub(r"don\'t",'do not',text)
        text=re.sub(r"don’t",'do not',text)


        text=re.sub(r"doesn\'t",'does not',text)
        text=re.sub(r"doesn’t",'does not',text)

        text=re.sub(r"hasn\'t",'has not',text)
        text=re.sub(r"hasn’t",'has not',text)

        text=re.sub(r"hasn\''",'has not',text)

        text=re.sub(r"hadn\'t",'had not',text)
        text=re.sub(r"hadn’t",'had not',text)


        text=re.sub(r"haven\'t",'have not',text)
        text=re.sub(r"haven’t",'have not',text)

        text=re.sub(r"haven\'",'have not',text)
        text=re.sub(r"haven’",'have not',text)

        text=re.sub(r"isn\'t",'is not',text)
        text=re.sub(r"isn’t",'is not',text)

        text=re.sub(r"isn\'",'is not',text)
        text=re.sub(r"isn’",'is not',text)


        text=re.sub(r"didn\'t",'did not',text)
        text=re.sub(r"didn’t",'did not',text)
        text=re.sub(r"didn’",'did not',text)

        text=re.sub(r"wasn\'t",'was not',text)
        text=re.sub(r"wasn’t",'was not',text)

        text=re.sub(r"wasn\'",'was not',text)
        text=re.sub(r"wasn’",'was not',text)

        text=re.sub(r"weren\'t",'were not',text)
        text=re.sub(r"weren’t",'were not',text)

        text=re.sub(r"weren\'",'were not',text)
        text=re.sub(r"weren’",'were not',text)

        text=re.sub(r"he\'s",'he is',text)
        text=re.sub(r"he’s",'he is',text)


        text=re.sub(r"\'ll",'will',text)
        text=re.sub(r"’ll",'will',text)

        text=re.sub(r"[-()\"/@/#;:<>{}+=~|?, ]"," ",text)
        text=re.sub(r'[^a-zA_Z]',' ',text)

        return text
    def get_wordnet_part_of_speech(self,word_tag):
        if word_tag.startswith('J'):
            return wordnet.ADJ
        elif word_tag.startswith('V'):
            return wordnet.VERB
        elif word_tag.startswith('N'):
            return wordnet.NOUN
        elif word_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN



# Modeling Using the Bag of Word Method (TFIDF)

# Defining a Class for Vectorizing the two columns and then Stack them Together (For Being Used in A PipeLine)

In [12]:


class TfidfVectorizer_two_columns(BaseEstimator,TransformerMixin):
    def __init__(self,max_features=300):
        self.max_features=max_features
    def fit(self,Data):
        self.Data=Data
        self.vectorizer_title=TfidfVectorizer(max_features=self.max_features,analyzer='word')
        Xtrain_title_v=self.vectorizer_title.fit_transform(self.Data.loc[:,'title'])
        self.vectorizer_text=TfidfVectorizer(max_features=self.max_features,analyzer='word')
        Xtrain_text_v=self.vectorizer_text.fit_transform(self.Data.loc[:,'text'])
        print(Xtrain_title_v)
        print(Xtrain_text_v)
        self.X_train_all=hstack([Xtrain_text_v,Xtrain_title_v])
        return self
    def fit_transform(self,Data,y=None):
        self.fit(Data)
        return self.X_train_all
    def transform(self,Data,y=None):
        Xtrain_title_v=self.vectorizer_title.transform(Data.loc[:,'title'])
        Xtrain_text_v=self.vectorizer_text.transform(Data.loc[:,'text'])
        X_train_all=hstack([Xtrain_text_v,Xtrain_title_v])        
        return X_train_all



In [34]:
train=pd.read_csv('train.csv')
missing_value_replacement=Missing_Value()
preprocessing_r=preprocessing()
train_preprocess=missing_value_replacement.fit_transform(train)
train_preprocess=preprocessing_r.fit_transform(train_preprocess)



Instance is Created
Instance is Created


G:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [53]:
#{'classifier':[xgboost.XGBClassifier()],
#         "classifier__learning_rate"    : [0.01, 0.10, 0.20, 0.30 ],
#         'classifier__min_child_weight': [1, 5, 10],
#         'classifier__gamma': [0.5, 1, 2, 5],
#         'classifier__subsample': [0.6, 0.8, 1.0],
#         'classifier__colsample_bytree': [0.6, 0.8, 1.0],
#         'classifier__max_depth': [3, 4, 5]
#         },
import time
t_first = time.time()

grid_param=[
        {'classifier':[GaussianNB()],
        'classifier__var_smoothing': np.logspace(0,-9, num=100)},
        {'classifier':[SVC()],
        'classifier__C': [0.1,1, 10, 100], 
        'classifier__gamma': [1,0.1,0.01,0.001],
        'classifier__kernel': ['rbf', 'poly', 'sigmoid']},
        {'classifier':[MultinomialNB()],
        'classifier__alpha': [0.01, 0.1, 0.5, 1.0, 10.0, ]}

]
pipeline = [ ('tfidf_two_vectors',TfidfVectorizer_two_columns()),('classifier',SVC())] 

pipe = Pipeline(pipeline) 
random_search=RandomizedSearchCV(pipe,grid_param,n_iter=5,cv=4,verbose=1)
fitted_model=random_search.fit(train_preprocess.iloc[:,:-1],train_preprocess.iloc[:,-1])
t_elapsed=time.time()-t_first

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


  (0, 2)	0.4285201320995098
  (0, 281)	0.4258709574221582
  (0, 251)	0.12661401289934154
  (0, 289)	0.32672382369179825
  (0, 298)	0.33608840314421523
  (0, 258)	0.34795186392891553
  (0, 221)	0.4202464372048337
  (0, 1)	0.318734805844134
  (1, 41)	0.6306529478556998
  (1, 297)	0.23439470302460524
  (1, 159)	0.22632531660904776
  (1, 114)	0.29289461047103027
  (1, 263)	0.29266051802323345
  (1, 124)	0.5698029501188401
  (2, 291)	0.2651280719462103
  (2, 75)	0.39709873255895445
  (2, 277)	0.33081586756031195
  (2, 294)	0.3402037584344309
  (2, 8)	0.3411463504104351
  (2, 13)	0.2698929941587068
  (2, 18)	0.37535175829116996
  (2, 214)	0.32347321900002546
  (2, 251)	0.12097953923719174
  (2, 289)	0.31218422623940684
  (3, 230)	0.7835154834608186
  :	:
  (15594, 163)	0.34839821212068267
  (15594, 263)	0.2597602631356526
  (15595, 39)	0.6486395418730196
  (15595, 285)	0.5915553289019738
  (15595, 79)	0.36257771748966916
  (15595, 263)	0.312836117394425
  (15596, 259)	0.4132297270735225
  (1

  (0, 122)	0.28968755223107406
  (0, 46)	0.4058401316879093
  (0, 221)	0.409696088607548
  (0, 69)	0.3887919683985962
  (0, 160)	0.25784859978218333
  (0, 62)	0.3140759015026739
  (0, 282)	0.36159471863630227
  (0, 110)	0.3694381098672872
  (1, 26)	0.3054148631164075
  (1, 169)	0.3089171296186003
  (1, 292)	0.5012231826854464
  (1, 21)	0.5385857645529054
  (1, 43)	0.3674695121793437
  (1, 106)	0.36736742015875634
  (2, 81)	0.5495942597979157
  (2, 298)	0.4518538720519098
  (2, 89)	0.47465422132974994
  (2, 253)	0.17080093200333604
  (2, 287)	0.4891877346270224
  (3, 19)	0.35603645163307823
  (3, 98)	0.5457438939315254
  (3, 116)	0.3505969198567988
  (3, 127)	0.6726688985912075
  (4, 58)	0.3752145755688552
  (4, 261)	0.1533308829523973
  :	:
  (15595, 276)	0.4866836785053664
  (15595, 241)	0.5103816315346473
  (15595, 30)	0.3393932160096083
  (15595, 165)	0.23497811704952518
  (15595, 261)	0.20727731091681284
  (15595, 253)	0.15106138942399838
  (15596, 289)	0.5384279699657797
  (15596,

  (0, 122)	0.2926819186329524
  (0, 49)	0.40813962621492134
  (0, 219)	0.40813962621492134
  (0, 70)	0.3876907002115837
  (0, 162)	0.2573075870513423
  (0, 64)	0.31484150119558435
  (0, 282)	0.36000715797631877
  (0, 110)	0.3687001321909377
  (1, 27)	0.3075439108609229
  (1, 172)	0.3134483460089795
  (1, 292)	0.5012255696444659
  (1, 22)	0.5328040555872572
  (1, 45)	0.3698983974442441
  (1, 105)	0.36774287148649876
  (2, 79)	0.5568753404518232
  (2, 298)	0.44837123258990863
  (2, 88)	0.4666456516117175
  (2, 251)	0.16929685289832125
  (2, 287)	0.492375369421615
  (3, 20)	0.35453496508067583
  (3, 98)	0.5495284823874768
  (3, 116)	0.3507796000199602
  (3, 127)	0.6702813422660646
  (4, 61)	0.4059115078156598
  (4, 260)	0.1675007240431896
  :	:
  (15594, 27)	0.30178959286234924
  (15594, 162)	0.3466400871373472
  (15595, 41)	0.6485420706818837
  (15595, 285)	0.5947115776373677
  (15595, 264)	0.31145801782910765
  (15595, 81)	0.35875510464617916
  (15596, 10)	0.607588994970865
  (15596, 25

...

  (0, 2)	0.4285201320995098
  (0, 281)	0.4258709574221582
  (0, 251)	0.12661401289934154
  (0, 289)	0.32672382369179825
  (0, 298)	0.33608840314421523
  (0, 258)	0.34795186392891553
  (0, 221)	0.4202464372048337
  (0, 1)	0.318734805844134
  (1, 41)	0.6306529478556998
  (1, 297)	0.23439470302460524
  (1, 159)	0.22632531660904776
  (1, 114)	0.29289461047103027
  (1, 263)	0.29266051802323345
  (1, 124)	0.5698029501188401
  (2, 291)	0.2651280719462103
  (2, 75)	0.39709873255895445
  (2, 277)	0.33081586756031195
  (2, 294)	0.3402037584344309
  (2, 8)	0.3411463504104351
  (2, 13)	0.2698929941587068
  (2, 18)	0.37535175829116996
  (2, 214)	0.32347321900002546
  (2, 251)	0.12097953923719174
  (2, 289)	0.31218422623940684
  (3, 230)	0.7835154834608186
  :	:
  (15594, 163)	0.34839821212068267
  (15594, 263)	0.2597602631356526
  (15595, 39)	0.6486395418730196
  (15595, 285)	0.5915553289019738
  (15595, 79)	0.36257771748966916
  (15595, 263)	0.312836117394425
  (15596, 259)	0.4132297270735225
  (1

  (0, 122)	0.28968755223107406
  (0, 46)	0.4058401316879093
  (0, 221)	0.409696088607548
  (0, 69)	0.3887919683985962
  (0, 160)	0.25784859978218333
  (0, 62)	0.3140759015026739
  (0, 282)	0.36159471863630227
  (0, 110)	0.3694381098672872
  (1, 26)	0.3054148631164075
  (1, 169)	0.3089171296186003
  (1, 292)	0.5012231826854464
  (1, 21)	0.5385857645529054
  (1, 43)	0.3674695121793437
  (1, 106)	0.36736742015875634
  (2, 81)	0.5495942597979157
  (2, 298)	0.4518538720519098
  (2, 89)	0.47465422132974994
  (2, 253)	0.17080093200333604
  (2, 287)	0.4891877346270224
  (3, 19)	0.35603645163307823
  (3, 98)	0.5457438939315254
  (3, 116)	0.3505969198567988
  (3, 127)	0.6726688985912075
  (4, 58)	0.3752145755688552
  (4, 261)	0.1533308829523973
  :	:
  (15595, 276)	0.4866836785053664
  (15595, 241)	0.5103816315346473
  (15595, 30)	0.3393932160096083
  (15595, 165)	0.23497811704952518
  (15595, 261)	0.20727731091681284
  (15595, 253)	0.15106138942399838
  (15596, 289)	0.5384279699657797
  (15596,

  (0, 122)	0.2926819186329524
  (0, 49)	0.40813962621492134
  (0, 219)	0.40813962621492134
  (0, 70)	0.3876907002115837
  (0, 162)	0.2573075870513423
  (0, 64)	0.31484150119558435
  (0, 282)	0.36000715797631877
  (0, 110)	0.3687001321909377
  (1, 27)	0.3075439108609229
  (1, 172)	0.3134483460089795
  (1, 292)	0.5012255696444659
  (1, 22)	0.5328040555872572
  (1, 45)	0.3698983974442441
  (1, 105)	0.36774287148649876
  (2, 79)	0.5568753404518232
  (2, 298)	0.44837123258990863
  (2, 88)	0.4666456516117175
  (2, 251)	0.16929685289832125
  (2, 287)	0.492375369421615
  (3, 20)	0.35453496508067583
  (3, 98)	0.5495284823874768
  (3, 116)	0.3507796000199602
  (3, 127)	0.6702813422660646
  (4, 61)	0.4059115078156598
  (4, 260)	0.1675007240431896
  :	:
  (15594, 27)	0.30178959286234924
  (15594, 162)	0.3466400871373472
  (15595, 41)	0.6485420706818837
  (15595, 285)	0.5947115776373677
  (15595, 264)	0.31145801782910765
  (15595, 81)	0.35875510464617916
  (15596, 10)	0.607588994970865
  (15596, 25

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 46.7min finished


  (0, 120)	0.2907350573921392
  (0, 47)	0.408767009220908
  (0, 221)	0.4095960045989716
  (0, 68)	0.3881874628587788
  (0, 162)	0.2581397071766378
  (0, 62)	0.31413612895448456
  (0, 282)	0.3601315651047521
  (0, 108)	0.3673035599487976
  (1, 26)	0.3068290546780394
  (1, 171)	0.31033520940889875
  (1, 292)	0.5019522588094848
  (1, 21)	0.5356788391336684
  (1, 44)	0.3682827346069793
  (1, 103)	0.36743424932548474
  (2, 78)	0.5533044829739939
  (2, 298)	0.44949602032631764
  (2, 87)	0.47319394575544105
  (2, 251)	0.16980842997863746
  (2, 287)	0.4889376889137008
  (3, 19)	0.3550181567401479
  (3, 96)	0.5479845793471605
  (3, 114)	0.35176243580496436
  (3, 125)	0.6707743271317871
  (4, 59)	0.4091097657837568
  (4, 260)	0.16670013142152518
  :	:
  (20794, 26)	0.3020619136996462
  (20794, 162)	0.3481512516002411
  (20795, 40)	0.6498710902010221
  (20795, 285)	0.5925396312723065
  (20795, 263)	0.3109366468771648
  (20795, 80)	0.3603924987030104
  (20796, 9)	0.6093041713792928
  (20796, 259)	

In [64]:
fitted_model.best_estimator_
# fitted_model.best_score_
# fitted_model.cv_results_

Pipeline(steps=[('tfidf_two_vectors', TfidfVectorizer_two_columns()),
                ('classifier', SVC(C=100, gamma=1))])

G:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [65]:
pipeline = [('missing_value', Missing_Value()), ('preprocess', preprocessing()),('tfidf_two_vectors',TfidfVectorizer_two_columns()),('classifier',SVC(C=100, gamma=1))] 

pipe = Pipeline(pipeline) 
pipe.fit(train.iloc[:,:-1],train.iloc[:,-1])



Instance is Created
Instance is Created


G:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


  (0, 120)	0.2908140037653311
  (0, 47)	0.4088780060616329
  (0, 222)	0.40970722654560254
  (0, 68)	0.3876799726274774
  (0, 163)	0.258209802588733
  (0, 62)	0.3141139042045069
  (0, 282)	0.36022935544758855
  (0, 108)	0.36740329777944847
  (1, 26)	0.3068290546780394
  (1, 172)	0.31033520940889875
  (1, 292)	0.5019522588094848
  (1, 21)	0.5356788391336684
  (1, 44)	0.3682827346069793
  (1, 103)	0.36743424932548474
  (2, 78)	0.5525451762171614
  (2, 298)	0.44976792062590004
  (2, 87)	0.47348018093838673
  (2, 251)	0.16991114715719954
  (2, 287)	0.489233447492378
  (3, 19)	0.35580496465204514
  (3, 96)	0.5491990485072368
  (3, 114)	0.3525070106392201
  (3, 125)	0.6689708810538044
  (4, 59)	0.4091097657837568
  (4, 260)	0.16670013142152518
  :	:
  (20794, 26)	0.3020619136996462
  (20794, 163)	0.3481512516002411
  (20795, 40)	0.6498710902010221
  (20795, 285)	0.5925396312723065
  (20795, 263)	0.3109366468771648
  (20795, 80)	0.3603924987030104
  (20796, 9)	0.6093041713792928
  (20796, 259)

Pipeline(steps=[('missing_value', Missing_Value()),
                ('preprocess', preprocessing()),
                ('tfidf_two_vectors', TfidfVectorizer_two_columns()),
                ('classifier', SVC(C=100, gamma=1))])

### Note that since we used a pipeline, the input for the test dataset is a pandas dataset. We even do not need to choose the columns as long as there are two columns with names 'title' and 'text' that would be sufficient

In [ ]:
test_set=pd.read_csv('test.csv')
y_predict_test=pipe.predict(test_set)



## Evaluation of the Resulting test dataset


In [67]:
from sklearn.metrics import accuracy_score
test_label=pd.read_csv('labels.csv')
y_test=test_label['label'].values
f1_score(y_test,y_predict_test)

0.6737410071942446